In [3]:
import pandas as pd
import logging
import argparse
from pathlib import Path
import numpy as np
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
#from wide_resnet import WideResNet
from utils import load_data
from keras.preprocessing.image import ImageDataGenerator
from mixup_generator import MixupGenerator
from random_eraser import get_random_eraser
from PIL import Image



import pandas as pd
import os
from PIL import Image
import numpy as np
from numpy import save, load
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, \
    GlobalMaxPool2D, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, Lambda
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras import backend as K


from tqdm import tqdm
from collections import Counter

In [4]:
logging.basicConfig(level=logging.DEBUG)

In [5]:
# input_path = "./Dataset-copy/imdb_faces/imdb_db.mat"
# image, gender, age, _, image_size, _ = load_data(input_path)

In [6]:
def read_and_resize(filepath, input_shape=(256, 256)):
    im = Image.open((filepath)).convert('RGB')
    im = im.resize(input_shape)
    im_array = np.array(im, dtype="uint8")#[..., ::-1]
    return np.array(im_array / (np.max(im_array)+ 0.001), dtype="float32")

In [7]:
datagen = ImageDataGenerator(
    rotation_range=6,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1)

In [8]:
def augment(im_array):
    im_array = datagen.random_transform(im_array)
    return im_array

In [9]:
def gen(df, batch_size=8, aug=False):
    df = df.sample(frac=1)

    dict_age = {'(0, 2)' : 0,
                '(3, 5)' : 1,
                '(6, 10)' : 2,
                '(11, 15)' : 3,
                '(16, 20)' : 4,
                '(21, 30)' : 5,
                '(31, 40)' : 6,
                '(41, 50)' : 7,
                '(51, 60)' : 8,
                '(61, 70)' : 9,
                '(71, 80)' : 10,
                 '(81, 90)' : 11,
                 '(91, 100)' : 12}

    while True:
        for i, batch in enumerate([df[i:i+batch_size] for i in range(0,df.shape[0],batch_size)]):
            if aug:
                images = np.array([augment(read_and_resize(file_path)) for file_path in batch.path.values])
            else:
                images = np.array([read_and_resize(file_path) for file_path in batch.path.values])


            #labels = np.array([dict_age[g] for g in batch.out_ages.values])
            labels = np.array(batch.out_ages.values)

            labels = labels[..., np.newaxis]

            yield images, labels

In [10]:
optimizer = Adam(lr=0.1)

In [11]:
def get_model(n_classes=1):

    base_model = ResNet50(weights="./resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5", include_top=False)

    #for layer in base_model.layers:
    #    layer.trainable = False

    x = base_model.output
    x = GlobalMaxPooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.5)(x)
    if n_classes == 1:
        x = Dense(n_classes, activation="sigmoid")(x)
    else:
        x = Dense(n_classes, activation="softmax")(x)

    base_model = Model(base_model.input, x, name="base_model")
    if n_classes == 1:
        base_model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=optimizer)
    else:
        base_model.compile(loss="sparse_categorical_crossentropy", metrics=['acc'], optimizer=optimizer)

    return base_model

In [12]:
if __name__ == "__main__":
    base_path = "./Dataset-copy/"

    dict_age = {'(0, 2)' : 0,
                '(3, 5)' : 1,
                '(6, 10)' : 2,
                '(11, 15)' : 3,
                '(16, 20)' : 4,
                '(21, 30)' : 5,
                '(31, 40)' : 6,
                '(41, 50)' : 7,
                '(51, 60)' : 8,
                '(61, 70)' : 9,
                '(71, 80)' : 10,
                 '(81, 90)' : 11,
                 '(91, 100)' : 12}

    bag = 3

    all_indexes = list(range(5))

    accuracies = []
    print("Reading train and test CSV files ")
    train_df = pd.read_csv("imdb_dataset.csv")
    #test_df = pd.read_csv("test_gender_filtered_data_with_path.csv")
    tr_tr, tr_val = train_test_split(train_df, test_size=0.1,random_state = 100)
    tr_val['out_ages'].groupby
    print("Reading Done.")
    cnt_ave = 0
    predictions = 0
#     print("Extracting test labels and test images from files")
#     test_images = load("imdb_test_images.npy")
#     test_labels = load("imdb_test_labels.npy")
#     print("Extracting Done.")
    #tr_tr, tr_val = train_test_split(train_df, test_size=0.1,random_state = 100)
    file_path = "imdb_age_recog_weights_intel_py.h5"
    
    print("Generating callback_list")
    
    checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

    #early = EarlyStopping(monitor="val_acc", mode="max", patience=5)

    reduce_on_plateau = ReduceLROnPlateau(monitor="val_loss",
                                          mode="min", 
                                          factor=0.1,
                                          #cooldown=0,
                                          patience=3,
                                          verbose=1,
                                          min_lr=0.00001)

    callbacks_list = [checkpoint,
                      reduce_on_plateau,
                      #early
                     ]  # early
    print("Done Generating callbacklist.")
    print("generating Model")
    model = get_model(n_classes=13)
    print("Done generating model")
    print("Running Fit_generator")
    model.fit_generator(gen(tr_tr,batch_size=8, aug=True), 
                        validation_data=gen(tr_val), 
                        epochs=200, 
                        verbose=1, 
                        #workers=4,
                        callbacks=callbacks_list,
                        steps_per_epoch=500,#int(10740.75), 
                        validation_steps=50,
                        #use_multiprocessing=True
                       )
    #model.save(file_path)
    print("Trained Model saved to disk")

Reading train and test CSV files 
Reading Done.
Generating callback_list
Done Generating callbacklist.
generating Model




/home/vamsik1211/anaconda3/envs/intelpython/lib/python3.5/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Done generating model
Running Fit_generator



Epoch 1/200
 51/500 [==>...........................] - ETA: 36:44 - loss: 281.4756 - acc: 0.2721

KeyboardInterrupt: 

In [7]:

train_df = pd.read_csv("imdb_dataset.csv")
tr_tr, tr_val = train_test_split(train_df, test_size=0.1,random_state = 100)
tr_val, tr_test = train_test_split(tr_val,test_size=0.1,random_state = 100)
test_images = np.array([read_and_resize(file_path) for file_path in tr_test.path.values])

In [11]:
save("imdb_test_images.npy",test_images)

In [13]:
test_labels = np.array([int(g == "m") for g in tr_test.out_ages.values])

In [14]:
save("imdb_test_labels.npy",test_images)